# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [3]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [4]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [5]:
"""print out the number of twits"""

# TODO Implement 

sum(len(i) for i in twits.values())

1548010

### Split Message Body and Sentiment Score

In [6]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [7]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'http\S+', ' ', text)
    text=json.dumps(text)
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text =  re.sub(r'\$\S+'," ",text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'\@\S+'," ",text)
    text=json.dumps(text)

    # Replace everything not a letter with a space
    text = re.sub(r"[^a-zA-Z]", " ", text)
    text = re.sub(r'[0-9]+', '', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(word, pos="v") for word in tokens if len(word)>1 ] 
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [8]:
# TODO Implement

tokenized =[preprocess(message) for message in messages]

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [9]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

bow = Counter()
for t in tokenized:
    bow.update(t)

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [10]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {word: count/len(tokenized) for word, count in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.000005

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 15

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [b[0] for b in bow.most_common(high_cutoff)]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['be', 'the', 'to', 'for', 'on', 'of', 'and', 'in', 'this', 'it', 'ud', 'at', 'will', 'up', 'buy']


20635

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [11]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab =  {word: i for i, word in enumerate(filtered_words, 1)} 
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {i: word for word, i in vocab.items()}
# tokenized with the words not in `filtered_words` removed.
filtered = []
check_values = set(filtered_words)
for token in tokenized:
    filt = [w for w in token if w in check_values]
    filtered.append(filt)

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [12]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [13]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19534714613975043

Finally let's convert our tokens into integer ids which we can pass to the network.

In [14]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']
 

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [15]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout=dropout, batch_first=False)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and softmax layers
        self.fc = nn.Linear(lstm_size, output_size)
        self.sig = nn.LogSoftmax(dim=1) 


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        #train_on_gpu = False
        if (train_on_gpu):
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
        else:
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden        


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        
        # TODO Implement 
        batch_size = nn_input.size(1) 
        

        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
  

        lstm_out, hidden = self.lstm(embeds, hidden_state)
    
        # stack up lstm outputs
        #lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        lstm_out = lstm_out[-1, :, :]
   
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        #sig_out = sig_out.view(batch_size, -1)
        #print(sig_out.shape)
        #sig_out = sig_out[:, -1] # get last batch of labels
        #print(sig_out.shape)
        
        # return last sigmoid output and hidden state
        return sig_out, hidden        

In [ ]:
# First checking if GPU is available
def check_gpu():
    train_on_gpu=torch.cuda.is_available()

    #if(train_on_gpu):
        #print('Training on GPU.')
    #else:
        #print('No GPU available, training on CPU.')
    return train_on_gpu
check_gpu()

In [17]:
train_on_gpu = False

### View Model

In [18]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2).cuda() 
model.embedding.weight.data.uniform_(-1, 1)


input = torch.randint(0, 1000, (5, 4), dtype=torch.int64).cuda() 
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps )

tensor([[-1.9174, -1.7117, -1.3036, -1.5909, -1.6239],
        [-1.9168, -1.6593, -1.3283, -1.6147, -1.6156],
        [-1.9457, -1.5286, -1.3743, -1.5749, -1.7136],
        [-1.9455, -1.5434, -1.3666, -1.5761, -1.7056]], device='cuda:0')


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [20]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [19]:
import numpy
# TODO Implement 
split_frac = 0.8
split_idx = int(split_frac * len(token_ids))

train_features = token_ids[:split_idx]
valid_features = token_ids[split_idx:]

train_labels = sentiments[: split_idx]
valid_labels = sentiments[split_idx:]
len(train_features)

825128

In [21]:
## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(len(train_features)), 
      "\nValidation set: \t{}".format(len(valid_features)))

			Feature Shapes:
Train set: 		825128 
Validation set: 	206282


In [22]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5,  1).cuda()
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch.cuda() , hidden)

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


### Training
It's time to train the neural network!

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5,  2,  0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(20636, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (sig): LogSoftmax()
)

In [22]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))

print('Sample input size: ', text_batch.size()) # seq_length, batch_size
print('Sample input: \n', text_batch)
print()
print('Sample label size: ', labels.size()) # batch_size
print('Sample label: \n', labels)

Sample input size:  torch.Size([20, 64])
Sample input: 
 tensor([[   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0],
        [   0,    0,    0,  ...,    0,    0,    0],
        ...,
        [   1,   14,   21,  ...,   71,   14,    0],
        [   2,   15,   25,  ...,   73,   15,  246],
        [   3,   16,   26,  ...,   74,   16,  247]])

Sample label size:  torch.Size([64])
Sample label: 
 tensor([ 4,  4,  3,  2,  3,  0,  3,  4,  0,  4,  0,  4,  4,  4,
         3,  4,  4,  3,  3,  3,  0,  1,  3,  3,  0,  4,  4,  0,
         3,  1,  2,  2,  3,  3,  4,  3,  2,  4,  3,  3,  2,  2,
         3,  3,  3,  3,  1,  4,  3,  4,  0,  3,  2,  0,  1,  4,
         1,  3,  4,  1,  2,  0,  4,  2])


In [35]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs =1
batch_size = 256
learning_rate = 0.003

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    value=0
    total=0
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        hidden = model.init_hidden(text_batch.shape[1])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        
        outputs,hidden = model(text_batch.cuda(),hidden)
        model.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, y = torch.max(outputs.data, 1)
        total += labels.size(0)
        value += (y == labels).sum().item()
 
        
        
        if steps % print_every == 0:
            model.eval()
            
            
            # TODO Implement: Print metrics
                       
    
            print("Steps: ",steps, "losses: ", loss.item(),  "accuracy: ",value / total)
            

    
            
            model.train()
print("done")

Starting epoch 1
Steps:  100 losses:  0.8375991582870483 accuracy:  0.6752734375
Steps:  200 losses:  0.8092561960220337 accuracy:  0.6741015625
Steps:  300 losses:  0.8140150904655457 accuracy:  0.6731510416666666
Steps:  400 losses:  0.8097780346870422 accuracy:  0.673828125
Steps:  500 losses:  0.8680776357650757 accuracy:  0.67325
Steps:  600 losses:  0.8987329006195068 accuracy:  0.6726302083333333
Steps:  700 losses:  0.846504807472229 accuracy:  0.6723493303571428
Steps:  800 losses:  0.7727811932563782 accuracy:  0.6720751953125
Steps:  900 losses:  0.8572314381599426 accuracy:  0.6719574652777778
Steps:  1000 losses:  0.7841613292694092 accuracy:  0.671296875
Steps:  1100 losses:  0.7265303134918213 accuracy:  0.6706889204545454
Steps:  1200 losses:  0.9758467674255371 accuracy:  0.6709928385416667
Steps:  1300 losses:  0.9048140645027161 accuracy:  0.6708473557692308
Steps:  1400 losses:  0.8927345275878906 accuracy:  0.6712025669642857
Steps:  1500 losses:  0.856487691402435

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [113]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in vocab]  
    # Convert words to ids
    tokens = {word: i for i, word in enumerate(tokens)} 
           
    # Adding a batch dimension
    model =TextClassifier(len(vocab), 12, 1, 5, 2, 0.1)   
    text_input = torch.randint(0, 10, (2, len(tokens)), dtype=torch.int64)
    # Get the NN output
    hidden = model.init_hidden(len(tokens))
    logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pl=print("predictions and labels ")
    
    pred = pl,logps ,hidden
    
    return pred

In [114]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

predictions and labels 


(None, tensor([[-2.0305, -1.5265, -2.2790, -0.8210, -2.2158],
         [-1.9246, -1.6239, -2.5755, -0.7104, -2.4153],
         [-1.9279, -1.6201, -2.5649, -0.7139, -2.4080],
         [-2.0305, -1.5265, -2.2790, -0.8210, -2.2158],
         [-1.9149, -1.6349, -2.6068, -0.7003, -2.4367],
         [-2.0305, -1.5265, -2.2790, -0.8210, -2.2158]]), (tensor([[[ 0.],
           [ 0.],
           [ 0.],
           [ 0.],
           [ 0.],
           [ 0.]],
  
          [[ 0.],
           [ 0.],
           [ 0.],
           [ 0.],
           [ 0.],
           [ 0.]]]), tensor([[[ 0.],
           [ 0.],
           [ 0.],
           [ 0.],
           [ 0.],
           [ 0.]],
  
          [[ 0.],
           [ 0.],
           [ 0.],
           [ 0.],
           [ 0.],
           [ 0.]]])))

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

The prediction of the model is being displayed as a single digit(label) after the predictions are being displayed so as to correlate with each prediction for an optimal accuracy or true value.

In the line of the uncertainty of the prediction, a mathematical model is a construction by which an output or prediction y is determined from a set of inputs x. Prediction uncertainty refers to the variability in prediction due to an arguably alternative input value.  
  

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [115]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [116]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [117]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [118]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

predictions and labels 
predictions and labels 
predictions and labels 


{'symbol': '$AAPL',
 'score': (None, tensor([[-0.9352, -1.8875, -1.8584, -1.8271, -1.9715],
          [-0.9353, -1.8855, -1.8550, -1.8325, -1.9709]]), (tensor([[[ 0.],
            [ 0.]],
   
           [[ 0.],
            [ 0.]]]), tensor([[[ 0.],
            [ 0.]],
   
           [[ 0.],
            [ 0.]]]))),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.